In [ ]:
!pip install pandas numpy nltk spacy fasttext
!python -m spacy download pl_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 25.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import re
import nltk
import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import download
from difflib import SequenceMatcher
from pathlib import Path
from functools import cache

In [ ]:
download("stopwords")
download("punkt")
nltk.download('punkt_tab')

nlp = spacy.load("pl_core_news_sm")

stop_words = set(stopwords.words("polish"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [34]:
df = pd.read_csv('BAN-PL.csv')

In [ ]:
df = df.dropna()

df = df.drop_duplicates()

In [36]:
df = df[["Text", "Class"]]
df.head(5)

,Text,Class
0,\n\n\n Polska wtedy oficjalnie powi...,0
1,\n Gigantyczna różnica\n\n{USERNAME}: biorac ...,0
2,\n\n {USERNAME}: Moj kumpel budowla...,0
3,"kura, rzodkiewka za 3pln to nie jest tanio",0
4,{USERNAME}: większość nie idzie w marszu za PO...,0


In [ ]:
df_raw = df.copy()
df_raw.to_csv('BAN-PL_raw.csv', index=False)

In [ ]:
MAIN_DIR = Path("/content")
DATA_DIR = MAIN_DIR
MODEL_DIR = MAIN_DIR

LINK_FILTER = r"(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])"
#LENNY_FACE_FILTER = r"[•́̀￣ݓ✖･՞﹒︣⌣⁰❛¯͒´`ཀ༎ຶຈO͡◕͠°⇀↼ಥ☯͝ಠೃರ˙◔□⌐▀・◉‾⊙◐◖◗◑ヘ¬≖̿ิʘ̆☉;■º͜’^–xᴼ＾˘۞◯๑ᵔσ✪♥ั╥ᵒ̌ಡ̲ᵕିୖଵ்౦್രිᓀᓂ⊡⊚⊘⊗¤☢⚆☭◪Ɵ].{0,4}[_ਊ︿〜〰∧Д۝ڡʖ͜ل‸⌂͟﹏益‿̯Ĺʟдᗜᴥω◞౪◟෴ुرں┏┓ヮ▽▃ₒ̀ε~□◡3◯̫╭╮()▾＊ⓞ].{0,4}[•́̀￣ݓ✖･՞﹒︣⌣]"
LENNY_FACE_FILTER3 = r"¯\\_(ツ)_/¯|༼ ͡° ͜ʖ ͡° ༽|\(╯ ͠° ͟ʖ ͡°\)╯┻━┻|\[̲̅\$\̲̅\(̲̅ ͡° ͜ʖ ͡°̲̅\)̲̅\$\̲̅\]|\( ͡° ͜ʖ ͡°\)|\( ͠° ͟ʖ ͡°\)|\( ͡~ ͜ʖ ͡°\)|\( ͡ʘ ͜ʖ ͡ʘ\)|\( ͡o ͜ʖ ͡o\)|\(° ͜ʖ °\)|\( ‾ʖ̫‾\)|\( ಠ ͜ʖಠ\)|\( ͡° ʖ̯ ͡°\)|\( ͡ಥ ͜ʖ ͡ಥ\)|\(▀̿Ĺ̯▀̿ ̿\)|\( ✧≖ ͜ʖ≖\)|\(ง ͠° ͟ل͜ ͡°\)ง|\(͡ ͡° ͜ つ ͡͡°\)|\(✿❦ ͜ʖ ❦\)|ᕦ\( ͡° ͜ʖ ͡°\)ᕤ|\( ͡° ͜ʖ ͡°\)╭∩╮|¯\\_\( ͡° ͜ʖ ͡°\)_/¯|\( ͡°\( ͡° ͜ʖ\( ͡° ͜ʖ ͡°\)ʖ ͡°\) ͡°\)|ಠ_ಠ|\(⌐ ͡■ ͜ʖ ͡■\)|\( ͡º ͜ʖ͡º\)|\( ͡° ͜ʖ ͡° \)つ──☆*:・ﾟ|\(╯°□°\）╯\(⇀‸↼‶\)|¯\\_\(ツ\)_/¯|\(－‸ლ\)|ᕦ òóˇ ᕤ"
#LENNY_FACE_FILTER = r'[^\w\s]'
LOOSE_COMBOS = r"""(?:[ʖつﾟ_ツʘᶘᵒᴥᵒᶅ]{1,3}\s*){2,}"""
EXTRA_EMOTE_REGEX = r"""
\([^()]*[͡ʖ°~つﾟ_ツʘᶘᵒᴥᵒᶅᴥ☆*:・┻╯╮ᕙᕗʕʔ]+\s*[^()]*\)
|ᶘᵒᴥᵒᶅ
|ʘ\s*ʘ
|_+\s*ツ\s*_+
|¯\\_\(ツ\)_/¯
"""
LOOSE_COMBOS = r"""(?:[ʖつﾟ_ツʘᶘᵒᴥᵒᶅˇòóᴥᕙᕗʕʔ]{1,3}\s*){2,}"""
MUSCLE_ARMS = r"ᕦ[^ᕦᕤ]{0,20}ᕤ"
ALT_MUSCLE_ARMS = r"ᕙ[^ᕙᕗ]{0,20}ᕗ"
BEAR_FACE = r"ʕ\s*ᴥ\s*ʔ"
LENNY_FACE_FILTER21 ="\([^()]*[͡ʖ°~つﾟ_ツʘᶘᵒᴥᵒᶅ☆*:・┻╯╮]\s*[^()]*\)|ᶘᵒᴥᵒᶅ|ʘ\s*ʘ|_+\s*ツ\s*_+|¯\\_\(ツ\)_/¯"
LENNY_FACE_FILTER2 = rf"""(
    {LENNY_FACE_FILTER3}
    |{EXTRA_EMOTE_REGEX}
    |{LOOSE_COMBOS}
    |{MUSCLE_ARMS}
    |{ALT_MUSCLE_ARMS}
    |{BEAR_FACE}
    |{LENNY_FACE_FILTER21}
)"""
MASKED_PROFANITY_REGEX = r"[A-Ża-ż]+(?:[\*+|\!+|\?+|\.+|#+|@+|\$+|%+|\^+|&+]{2,}[A-Ża-ż]*)?"
NICKNAMES_FILTER = r"\*\*.+\*\*: |@\S+: | @\S+|\{USERNAME\}: |\{USERNAME\}|\[surname\]|\[pseudonym\]|\{URL}|\[address]|\[email]|\[phonenumber]|\[number]"
ONE_CHANGED_REGEX = r"[a-żA-Ż]+[8|4|3|@|q|!|$|0|1]+[a-żA-Ż|8|4|3|@|!|$|0|1]*"
POPULAR_SIGNS = r"[8|4|3|@|q|!|$|0|1]"

POPULAR_CHANGES = {
    "i": ["&", "1", "!"],
    "a": ["@", "4"],
    "e": ["3"],
    "l": ["1", "!"],
    "s": ["$", "5"],
    "ś": ["$"],
    "o": ["0"],
    "z": ["2"],
    "b": ["8"],
    "ku": ["q"],
    "k": ["q"],
}

In [ ]:
PROF_DF = pd.read_csv(DATA_DIR / "polish_vulgarisms_extended.txt", names=["profanity"])
PROFANITY_REGEX = "|".join(PROF_DF.values.flatten())

def similar(a: str, b: str) -> float:
    return SequenceMatcher(None, a, b).ratio()

def get_profanity(prof_df: pd.DataFrame, masked_word: str) -> str:

    CENSOR_SIGNS = r"[!@#$%^&*()+_=\\|<>?/;:'\"`~,]"

    masked_word_lower = masked_word.lower()
    prof_df["sim"] = prof_df["profanity"].apply(lambda x: similar(x.lower(), masked_word_lower))  
    alternative_words = prof_df.sort_values("sim", ascending=False)


    masked_clean = re.sub(r"[^a-żA-Ż]", "", masked_word_lower)
    num_censored = len(re.findall(CENSOR_SIGNS, masked_word))

    # Dopasowanie do wzorca start, middle, end
    match = re.match(r"([a-żA-Ż]+)([^a-żA-Ż]+)([a-żA-Ż]*)", masked_word_lower)
    if match:
        start, middle, end = match.groups()
    else:
        start, middle, end = masked_clean, "", ""

    # Nowy filtr: tylko słowa o tej samej długości co masked_word
    expected_length = len(start) + len(middle) + len(end)
    alternative_words = alternative_words[alternative_words["profanity"].str.len() == expected_length]

    # Filtrujemy słowa na podstawie start i end
    alternative_words["is_ok_start"] = alternative_words["profanity"].apply(lambda x: x.startswith(start))
    alternative_words["is_ok_end"] = alternative_words["profanity"].apply(lambda x: x.endswith(end))

    # Wybieramy najlepszą opcję
    both_ok = alternative_words[alternative_words["is_ok_start"] & alternative_words["is_ok_end"]]
    start_ok = alternative_words[alternative_words["is_ok_start"]]

    if not both_ok.empty:
        return both_ok.iloc[0, 0]
    elif not start_ok.empty:
        return start_ok.iloc[0, 0]
    else:
        raise ValueError("Niepoprawne wartości lub brak w słowniku.")

@cache
def replace_word(unknown: str, words: list) -> str:
    symbols = re.findall(POPULAR_SIGNS, unknown)
    options = {item: [key for key in POPULAR_CHANGES if item in POPULAR_CHANGES[key]] for item in symbols}
    start = unknown[: unknown.find(symbols[0])]
    end = unknown[unknown.find(symbols[-1]) + 1 :]

    sim_words = [word for word in words if word.startswith(start) and word.endswith(end) and len(word) == len(unknown)]

    if not sim_words:
        return unknown
    elif len(sim_words) == 1:
        return sim_words[0]

    potencial_words = [unknown.replace(symbols[i], option[i]) for option in itertools.product(*options.values()) for i in range(len(option))]
    exact_words = list(set(sim_words).intersection(potencial_words))

    if not exact_words:
        df = pd.DataFrame({"words": sim_words})
        for word in potencial_words:
            df[word] = df["words"].apply(lambda x: similar(x, word))
        return df.idxmax(axis=1)[0]
    return exact_words[0]

def unmask_profanities(text: str) -> str:
    prof_in_text = re.findall(MASKED_PROFANITY_REGEX, text)

    CENSOR_SIGNS = r"[!@#$%^&*?]"

    # Filtrujemy słowa, które zawierają przynajmniej jeden znak cenzurujący
    prof_in_text = [word for word in prof_in_text if re.search(CENSOR_SIGNS, word)]

    for item in prof_in_text:
        try:
            # Sprawdzamy, czy możemy znaleźć zamiennik
            replacement = get_profanity(PROF_DF, item)
            text = text.replace(item, replacement)
        except ValueError:
            continue

    return text

def clean_text(text: str, masked_profanity=False, filter_link=False, filter_lennyface=False, filter_hashtags=False, filter_nicknames=False) -> str:
    source = text.lower().find("źródło:")
    if source != -1:
        text = text[:source]
    if filter_link:
        text = re.sub(LINK_FILTER, " ", text).replace("{URL}", " ")
    if filter_hashtags:
        text = re.sub(r"#\S+", " ", text)
    if masked_profanity:
        text = unmask_profanities(text)
    if filter_nicknames:
        text = re.sub(NICKNAMES_FILTER, " ", text)
    if filter_lennyface:
        text = re.sub(LENNY_FACE_FILTER3, " ", text)
        #text = re.sub(LENNY_FACE_FILTER, " ", text)
        text = re.sub(r'[^\w\s\.\?]', '', text)
        text = re.sub(LENNY_FACE_FILTER2, " ", text)
    return " ".join(text.split())

In [ ]:
df['cleaned_text'] = df['Text'].apply(lambda x: clean_text(x,
                                                           masked_profanity=True,
                                                           filter_lennyface=True,
                                                           filter_link=True,
                                                           filter_hashtags=True,
                                                           filter_nicknames=True))

<ipython-input-42-86d3e3694ff1>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alternative_words["is_ok_start"] = alternative_words["profanity"].apply(lambda x: x.startswith(start))
<ipython-input-42-86d3e3694ff1>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alternative_words["is_ok_end"] = alternative_words["profanity"].apply(lambda x: x.endswith(end))
<ipython-input-42-86d3e3694ff1>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [44]:
df.tail(10000)

,Text,Class,cleaned_text
13999,{USERNAME}: zabij sie typie,1,zabij sie typie
14000,"{USERNAME}: ...nie będziesz mi mówił ""mentalny...",1,...nie będziesz mi mówił mentalny faszysto jak...
14001,Jebany jaka bramka \n#mecz,1,Jebany jaka bramka
14002,{USERNAME}: Czyli raptem nagrywanie bez zgody ...,1,Czyli raptem nagrywanie bez zgody innych jest ...
14003,"{USERNAME}: tak tak, tak to sobie tłumacz xDDD...",1,tak tak tak to sobie tłumacz xDDDDDDDDDD wypie...
...,...,...,...
23994,{USERNAME}: ale kurwa knur,1,ale kurwa knur
23995,Polaczki jebane XD na wszystko narzekać byle n...,1,Polaczki jebane XD na wszystko narzekać byle n...
23996,jakaś muzyka normicka dudni ehhh w sumie i tak...,1,jakaś muzyka normicka dudni ehhh w sumie i tak...
23997,"{USERNAME}: ""[pseudonym]"" czy raczej zjeb ze s...",1,czy raczej zjeb ze stawu to była jedna z najgo...


In [62]:
df['processed_text'].apply(lambda x: pd.isnull(x) or str(x).strip() == "").sum()

np.int64(0)

In [ ]:
df_selected = df[['Class', 'cleaned_text']].copy()

df_selected = df_selected.rename(columns={'cleaned_text': 'Text'})

df_selected.to_csv('BAN-PL_light.csv', index=False)

In [54]:
df['cleaned_text'] = df['cleaned_text'].str.lower()

In [ ]:
def process_text(text):
    words = word_tokenize(text.lower())

    words = [word for word in words if word not in stop_words and word.isalnum()]

    doc = nlp(" ".join(words))
    lemmatized_words = [token.lemma_ for token in doc]

    return " ".join(lemmatized_words)

In [56]:
df["processed_text"] = df["cleaned_text"].apply(process_text)

In [ ]:
df.tail(1000)

,Text,Class,cleaned_text,processed_text
22999,No kurwa co za pojeb przez pół godziny była ja...,1,no kurwa co za pojeb przez pół godziny była ja...,kurwa pojeb pół godzina jebać rozgrzewka chuj ...
23000,{USERNAME}: Socjal zawsze idzie w górę i zawsz...,1,socjal zawsze idzie w górę i zawsze będzie. a ...,socjal iść góra zostać pracować emerytura dost...
23001,"Witam, jestem ojcem [pseudonym] i piszę w tej ...",1,witam jestem ojcem i piszę w tej chwili z jego...,witać ojciec piszę chwila konto prosić żony wy...
23002,{USERNAME}: Dziadek z UPA pewnie zdementował c...,1,dziadek z upa pewnie zdementował cnie? 3,dziadek upa pewnie zdementować cni 3
23003,{USERNAME}: Idź być idiotą gdzieś indziej. Tut...,1,idź być idiotą gdzieś indziej. tutaj dorośli r...,iść idiota indziej dorosły rozmawiać
...,...,...,...,...
23994,{USERNAME}: ale kurwa knur,1,ale kurwa knur,kurwa knur
23995,Polaczki jebane XD na wszystko narzekać byle n...,1,polaczki jebane xd na wszystko narzekać byle n...,polaczka jebane xd narzekać byle narzekać band...
23996,jakaś muzyka normicka dudni ehhh w sumie i tak...,1,jakaś muzyka normicka dudni ehhh w sumie i tak...,muzyka normicki dudny ehhh suma mieć być iść s...
23997,"{USERNAME}: ""[pseudonym]"" czy raczej zjeb ze s...",1,czy raczej zjeb ze stawu to była jedna z najgo...,raczej zjeb staw najgorszy kreatur pojawić tot...


In [ ]:
df_selected = df[['Class', 'processed_text']].copy()

df_selected = df_selected.rename(columns={'processed_text': 'Text'})

df_selected.to_csv('BAN-PL_full.csv', index=False)